In [2]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import pandas as pd
import time, sys

In [3]:
def parse_html(box_score):
    try:
        with open(box_score, encoding="utf-8") as f:
            html = f.read()
    except UnicodeDecodeError:
        with open(box_score, encoding="latin1") as f:
            html = f.read()
    
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [4]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team","total"]]
    
    return line_score

In [5]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="ignore")
    if stat == "basic":
        df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
        df["+/-"] = df["+/-"].apply(lambda x: x.replace("+","") if isinstance(x, str) else x)
    else:
        df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]

    df["MP"] = df["MP"].apply(convert_to_decimal_minutes)
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [6]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [7]:
# Function to convert "MM:SS" format to decimal minutes
def convert_to_decimal_minutes(time_str):
    if ":" in time_str:
        minutes, seconds = map(int, time_str.split(":"))
        decimal_minutes = minutes + seconds / 60
        return decimal_minutes
    else:
        return time_str

In [403]:
def get_file_stats(box_scores):
    
    games_stats = []
    players_stats = []
    
    for box_score in box_scores:

        soup = parse_html(box_score)
        line_score = read_line_score(soup)
        teams = list(line_score["team"])
        player_summaries = []
        game_summaries = []
        base_cols = None

        for team in teams:
            basic = read_stats(soup, team, "basic")
            advanced = read_stats(soup, team, "advanced")
            summary = pd.concat([basic, advanced], axis=1)
            summary.columns = summary.columns.str.lower()
            maxes = summary.loc[summary.index != "Team Totals"].max()
            maxes.index = maxes.index.str.lower() + "_max"

            game =  pd.concat([summary.loc["Team Totals"], maxes], axis=0).to_frame().T

            summary["team"] = team
            game["team"] = team

            summary["team_opp"] = line_score.loc[line_score["team"] != team]["team"].iloc[0]
            game["team_opp"] = line_score.loc[line_score["team"] != team]["team"].iloc[0]

            summary["home"] = 0 if team == teams[0] else 1
            game["home"] = 0 if team == teams[0] else 1

            summary["won"] = 1 if line_score.loc[line_score["team"] == team]["total"].iloc[0] > line_score.loc[line_score["team"] != team]["total"].iloc[0] else 0
            game["won"] = 1 if line_score.loc[line_score["team"] == team]["total"].iloc[0] > line_score.loc[line_score["team"] != team]["total"].iloc[0] else 0

            if base_cols is None:
                base_players_cols = summary.columns.drop_duplicates(keep="first")
                base_game_cols = game.columns.drop_duplicates(keep="first")

                base_game_cols = [b for b in base_game_cols if "bpm" not in b and "+/-" not in b]
                base_players_cols = [b for b in base_players_cols if "bpm" not in b]


            summary = summary[base_players_cols].loc[:, ~summary[base_players_cols].columns.duplicated()]
            game = game[base_game_cols].loc[:, ~game[base_game_cols].columns.duplicated()]


            player_summaries.append(summary.loc[summary.index != "Team Totals"])
            game_summaries.append(game) 

        player_summary = pd.concat(player_summaries)
        game_summary = pd.concat(game_summaries)

        columns_to_keep = [col for col in game_summary.columns if "team" not in col]

        game_summary_opp = game_summary[columns_to_keep].iloc[::-1]
        game_summary_opp.columns += "_opp"

        full_game = pd.concat([game_summary, game_summary_opp], axis=1)
        full_game["season"] = read_season_info(soup)
        full_game["date"] = os.path.basename(box_score)[5:13]
        full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
        player_summary["season"] = read_season_info(soup)
        player_summary["date"] = os.path.basename(box_score)[5:13]
        player_summary["date"] = pd.to_datetime(player_summary["date"], format="%Y%m%d")

        games_stats.append(full_game)
        players_stats.append(player_summary)
        
        if len(games_stats)%1000 == 0:
            print(f"{len(games_stats)} of {len(box_scores)}")
    
    return games_stats, players_stats

In [ ]:
data_dir = "C:\\Users\\dfrv9\\Documents\\NBA"
csv_dir = os.path.join(data_dir, "csv files")
os.makedirs(csv_dir, exist_ok=True)
seasons = list(range(2016,2024))

for season in seasons:
    season_scores_dir = os.path.join(data_dir, "season scores", str(season))

    box_scores = os.listdir(season_scores_dir)
    box_scores = [os.path.join(season_scores_dir, f) for f in box_scores if f.endswith(".html")]

    games_stats, players_stats = get_file_stats(box_scores)
    
    games_stats_df = pd.concat(games_stats, ignore_index=True)
    players_stats_df = pd.concat(players_stats)

    save_path_games = os.path.join(csv_dir, f"nba_stats_games_{season}.csv")
    save_path_players = os.path.join(csv_dir, f"nba_stats_players_{season}.csv")

    players_stats_df.index.name = "Player"

    games_stats_df.to_csv(save_path_games, index=False)
    players_stats_df.to_csv(save_path_players)

1000 of 1316


In [398]:
games_stats_df.head()

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,mp_max,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,team_opp,home,won,mp_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,mp_max_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,home_opp,won_opp,season,date
0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,37.150000,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.0,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,CHI,0,0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,100.0,97.5,95.5,36.633333,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,1,1,2016,2015-10-27
1,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,100.0,97.5,95.5,36.633333,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,CLE,1,1,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,37.150000,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.0,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,0,0,2016,2015-10-27
2,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,36.0,59.0,23.0,5.0,3.0,15.0,15.0,106.0,0.493,0.448,0.302,0.271,41.1,83.7,59.6,62.2,5.2,5.5,12.3,100.0,111.2,98.6,37.150000,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.0,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,ATL,0,1,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,100.0,98.6,111.2,36.000000,8.0,16.0,1.000,2.0,6.0,1.000,3.0,4.0,1.0,3.0,7.0,8.0,4.0,4.0,3.0,5.0,4.0,20.0,1.330,1.500,1.000,2.000,20.5,28.8,17.9,34.6,10.6,7.1,57.1,33.8,258.0,121.0,ATL,1,0,2016,2015-10-27
3,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,100.0,98.6,111.2,36.000000,8.0,16.0,1.000,2.0,6.0,1.000,3.0,4.0,1.0,3.0,7.0,8.0,4.0,4.0,3.0,5.0,4.0,20.0,1.330,1.500,1.000,2.000,20.5,28.8,17.9,34.6,10.6,7.1,57.1,33.8,258.0,121.0,ATL,DET,1,0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,36.0,59.0,23.0,5.0,3.0,15.0,15.0,106.0,0.493,0.448,0.302,0.271,41.1,83.7,59.6,62.2,5.2,5.5,12.3,100.0,111.2,98.6,37.150000,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.0,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,0,1,2016,2015-10-27
4,240.0,35.0,83.0,0.422,6.

In [340]:
game.columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb',
       'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', '+/-', 'mp',
       'ts%', 'efg%', '3par', 'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%',
       'blk%', 'tov%', 'usg%', 'ortg', 'drtg', 'bpm', 'mp_max', 'fg_max',
       'fga_max', 'fg%_max', '3p_max', '3pa_max', '3p%_max', 'ft_max',
       'fta_max', 'ft%_max', 'orb_max', 'drb_max', 'trb_max', 'ast_max',
       'stl_max', 'blk_max', 'tov_max', 'pf_max', 'pts_max', '+/-_max',
       'mp_max', 'ts%_max', 'efg%_max', '3par_max', 'ftr_max', 'orb%_max',
       'drb%_max', 'trb%_max', 'ast%_max', 'stl%_max', 'blk%_max', 'tov%_max',
       'usg%_max', 'ortg_max', 'drtg_max', 'bpm_max', 'team', 'team_opp',
       'home', 'won'],
      dtype='object')